### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load,encoding="utf-8")
student_data = pd.read_csv(student_data_to_load,encoding="utf-8")

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [ ]:
# Convert school_data_complete into DataFrame
sc_df = pd.DataFrame(school_data_complete)
sc_df.head()

In [ ]:
#Create new variables for summary table

#Total number of Schools
total_schools = len(sc_df["school_name"].unique())

#Total number of students
total_students = len(sc_df["Student ID"].unique())

#Total Budget
# total_budget = sc_df["budget"].sum()
# budget_values = sc_df[["school_name","budget"]].values
budget_values = pd.unique(sc_df[["budget"]].values.ravel())
total_budget = sum(budget_values)

#Average Maths Score
ave_maths_score = sc_df["maths_score"].mean()

#Calculate the average reading score
ave_read_score = sc_df["reading_score"].mean()

#Calculate the percentage of students with a passing maths score (50 or greater)
perc_maths_pass = len(sc_df.loc[sc_df["maths_score"]>=50])/\
                        len(sc_df["maths_score"])*100

#Calculate the percentage of students with a passing reading score (50 or greater)
perc_read_pass =len(sc_df.loc[sc_df["reading_score"]>=50])/\
                        len(sc_df["reading_score"])*100

# Calculate the percentage of students who passed maths and reading (% Overall Passing)
perc_over_pass = len(sc_df.loc[(sc_df["maths_score"]>=50)&(sc_df["reading_score"]>=50)])/\
                            len(sc_df["reading_score"])*100


# Create a dataframe to hold the above results
summary_sc_df = pd.DataFrame({"Total Schools":[total_schools],
                                 "Total Students":[total_students],
                                  "Total Budget":[total_budget],
                                  "Average Maths Score":[ave_maths_score],
                                  "Average Reading Scor":[ave_read_score],
                                  "% Passing Maths":[perc_maths_pass],
                                  "% Passing Reading":[perc_read_pass],
                                  "% Passing Overall":[perc_over_pass],
                             })

summary_sc_df['Total Budget'] = summary_sc_df['Total Budget'].apply(lambda x: "${:,.2f}".format((x)))

summary_sc_df
# Optional: give the displayed data cleaner formatting


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [ ]:
#arrange columns of interest for data frame 

arrange_df =pd.DataFrame(sc_df[["school_name", "type", "Student ID", "budget", "reading_score", "maths_score","year","size", "student_name"]])
arrange_df['pass_reading'] = np.where(arrange_df['reading_score']>= 50, 1, 0)
arrange_df['pass_math'] = np.where(arrange_df['maths_score']>= 50, 1, 0)
arrange_df['pass_overall'] = np.where((arrange_df['maths_score']>= 50) & (arrange_df['reading_score']>= 50), 1, 0)
arrange_df['student_count'] = np.where(arrange_df['Student ID']!= '', 1, 0)
mapping_0 ={arrange_df.columns[0]:'School Name',arrange_df.columns[1]:'Type'}
arrange_df = arrange_df.rename(columns=mapping_0)
arrange_df.reset_index
arrange_df







In [ ]:
#Step 1 AGGREGATION
#Aggregate arranged data frame by school and type
school_group = arrange_df.groupby(["School Name","Type"])

#Step 2 : Agg at Sum
# items to be aggregated at sum: pass_math, pass_reading pass_overall, student count

aggsum_school_group = pd.DataFrame(school_group[["student_count","pass_reading","pass_math","pass_overall"]].sum())

# add in % Passing columns to df
aggsum_school_group["% Passing Maths"] = aggsum_school_group["pass_math"]/aggsum_school_group["student_count"]
aggsum_school_group["% Passing Reading"] = aggsum_school_group["pass_math"]/aggsum_school_group["student_count"]
aggsum_school_group["% Overall Passing"] = aggsum_school_group["pass_overall"]/aggsum_school_group["student_count"]

# aggsum_school_group.head()

#------------------
#Step 3 : Agg at mean
# items to be aggregated by mean: budget , reading_Score, maths_score

aggmean_school_group = pd.DataFrame(school_group[["budget","reading_score","maths_score"]].mean())
# aggmean_school_group.head()


In [137]:
# Merge Step 2 & Step 3 

school_group_merge = pd.merge(aggmean_school_group, aggsum_school_group, how="left", on=["School Name", "Type"])

# create field Per Student Budget
school_group_merge["Per Student Budget"] = school_group_merge["budget"]/school_group_merge["student_count"]
# school_group_merge.head()

school_group_summary = school_group_merge.iloc[:,[3,0,10,2,1,7,8,9]] # Select columns by Index
sg_summary_df = pd.DataFrame(school_group_summary)
sg_summary_df['Per Student Budget'] = sg_summary_df['Per Student Budget'].apply(lambda x: "${:,.2f}".format((x)))
sg_summary_df['budget'] = sg_summary_df['budget'].apply(lambda x: "${:,.2f}".format((x)))

#rename columns

mapping ={sg_summary_df.columns[0]:'Total Students',sg_summary_df.columns[1]:'Budget',sg_summary_df.columns[3]:'Average Maths Score',sg_summary_df.columns[4]:'Average Reading Score'}
sg_summary_df = sg_summary_df.rename(columns=mapping)
sg_summary_df.head(20)



,,Total Students,Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Name,Type,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,0.916399,0.916399,0.800844
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,0.908504,0.908504,0.807858
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,0.816548,0.816548,0.676501
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,0.824388,0.824388,0.674699
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,0.912125,0.912125,0.813351
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,0.809493,0.809493,0.663646
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,0.899297,0.899297,0.789227
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,0.816935,0.816935,0.667124
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,0.820626,0.820626,0.671918


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Year

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type